# 画像レジストレーション：位置合わせの技術

この章では、画像の**レジストレーション（Registration）**というトピックを扱います。レジストレーションとは、2つの画像に互いに重なるべき部分が含まれているにもかかわらず、実際にはずれてしまっている場合に行う「位置合わせ」の処理です。

例えば、同じ人物の写真を2枚撮影した場合を考えてみましょう。1枚目と2枚目の間でカメラが動いたり、被写体自身が動いたりすると、人物はそれぞれの写真のフレーム内で少し異なる位置に写ります。あるいは、ズームインすれば、人物の顔が写真内でより大きなスペースを占めることになります。背景に写っている他の部分も同様に移動する可能性があります。

まずは、この例を見てみましょう。ほぼ同じタイミングで撮影された2枚の写真を読み込みます。撮影の間にカメラと被写体が動いたため、2つの画像は少し異なって見えます。画像を並べて表示するだけでも違いは明らかですが、2つの画像から「ステレオ画像」を作成すると、その差はさらに明確になります。これは、RGB画像の赤チャンネルに1枚目の画像を、緑チャンネルに2枚目の画像を割り当てることで作成します。このステレオ画像（最も右側）では、1枚目の写真の内容が赤で、2枚目の写真の内容が緑で表示されます。両方の画像で内容が一致している部分は、これらの色が混ざり合って黄色に見えます。

In [ ]:
# ndslibのインストール（初回のみ）
!pip install ndslib

# 必要なライブラリのインポートと初期設定
from ndslib.config import jupyter_startup
jupyter_startup()

from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np

# 画像ファイルのダウンロード
!wget -q https://neuroimaging-data-science.org/_images/naomi1_xform.png
!wget -q https://neuroimaging-data-science.org/_images/naomi2_xform.png

naomi1 = imread('naomi1_xform.png')
naomi2 = imread('naomi2_xform.png')

# 画像の表示
fig, axes = plt.subplots(1, 3, figsize=(12, 5))
ax = axes.ravel()

ax[0].set_title('画像1 (固定画像)')
ax[0].imshow(naomi1)
ax[1].set_title('画像2 (移動画像)')
ax[1].imshow(naomi2)

# ステレオ画像の作成と表示
stereo = np.zeros((800, 600, 3), dtype=np.uint8)
stereo[..., 0] = naomi1
stereo[..., 1] = naomi2
ax[2].set_title('重ね合わせ (レジストレーション前)')
ax[2].imshow(stereo)

fig.tight_layout()
plt.show()

現状では、重ね合わせた画像で重なるべき部分が大きくずれてしまっています。これらの部分を重ね合わせるためには、一方の画像を変形させる必要があります。この変形には、画像の平行移動、拡大・縮小、回転、そしてせん断（シア）など、さまざまな種類があります。

この技術は、特に脳神経科学の分野で重要です。例えば、fMRI実験では、脳機能（例：BOLD信号）と脳構造（例：T1強調画像）という異なる側面のデータを収集します。もし被験者がこれらのデータを取得する間に頭を動かしてしまったら、どうなるでしょうか？このような場合に、画像レジストレーションを用いて画像間のずれを計算し、それらを正確に重ね合わせるのです。

ここでは、自由度の異なる3つの代表的なレジストレーション手法を学び、その違いを実際に見ていきましょう。

1.  **剛体変換 (Rigid Transformation)**: 平行移動と回転のみを扱います。物体の形や大きさは変わりません。最も制約の強い変換です。
2.  **アフィン変換 (Affine Transformation)**: 剛体変換に加え、拡大・縮小とせん断（シア）を許します。平行な線は変換後も平行なままです。
3.  **非線形変換 (Non-linear/Diffeomorphic Transformation)**: 画像の各部分が異なる動きをすることを許す、最も自由度の高い変換です。これにより、局所的な形状の違いにも対応できます。

## 実践1: 写真のレジストレーション

まずは先ほどの写真を使って、3つの変換手法を試してみましょう。ここでは、Pythonの強力な医用画像処理ライブラリである **DIPY (Diffusion Imaging in Python)** を使用します。

In [ ]:
# DIPYのインストール（初回のみ）
!pip install dipy

from dipy.align.imaffine import AffineRegistration
from dipy.align.transforms import RigidTransform2D, AffineTransform2D

### 1. 剛体変換 (Rigid Transformation)

最初に、平行移動と回転のみを許す剛体変換を試します。画像全体の向きと大まかな位置を合わせます。

In [ ]:
# レジストレーションの実行
affreg = AffineRegistration()

# 剛体変換モデルを定義
transform_rigid = RigidTransform2D()

# 最適化を実行して、変換パラメータを計算
rigid = affreg.optimize(naomi1, naomi2, transform_rigid, params0=None)

# 画像2に剛体変換を適用
naomi2_rigid = rigid.transform(naomi2)

# 結果の可視化
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
ax = axes.ravel()

ax[0].set_title('画像2 (剛体変換後)')
ax[0].imshow(naomi2_rigid)

stereo_rigid = np.zeros_like(stereo)
stereo_rigid[..., 0] = naomi1
stereo_rigid[..., 1] = naomi2_rigid
ax[1].set_title('重ね合わせ (剛体変換)')
ax[1].imshow(stereo_rigid)

fig.tight_layout()
plt.show()

剛体変換によって、画像全体の位置と回転が補正されました。しかし、黄色く重なっている部分を見ると、まだズームレベルの違いや、顔のパーツの微妙なずれが残っていることがわかります。

### 2. アフィン変換 (Affine Transformation)

次に、拡大・縮小とせん断を追加したアフィン変換を適用します。これにより、カメラのズームや傾きによる変形を補正できる可能性があります。

In [ ]:
# アフィン変換モデルを定義
transform_affine = AffineTransform2D()

# 最適化を実行
affine = affreg.optimize(naomi1, naomi2, transform_affine, params0=None)

# 画像2にアフィン変換を適用
naomi2_affine = affine.transform(naomi2)

# 結果の可視化
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
ax = axes.ravel()

ax[0].set_title('画像2 (アフィン変換後)')
ax[0].imshow(naomi2_affine)

stereo_affine = np.zeros_like(stereo)
stereo_affine[..., 0] = naomi1
stereo_affine[..., 1] = naomi2_affine
ax[1].set_title('重ね合わせ (アフィン変換)')
ax[1].imshow(stereo_affine)

fig.tight_layout()
plt.show()

アフィン変換の結果、剛体変換よりも背景や肩の位置などがより良く重なるようになりました。画像全体に適用されるグローバルな変形は、かなりうまく補正されています。しかし、被写体の顔の部分をよく見ると、まだ赤や緑のずれが残っています。これは、2枚の写真の間で被写体が頭を少し動かしたためであり、画像全体に一様な変換をかけるアフィン変換では捉えきれない局所的な変形です。

### 3. 非線形（Diffeomorphic）変換

最後に、局所的な変形を許すDiffeomorphic（微分同相写像）レジストレーションを試します。この手法は、画像の各部分が滑らかに変形することを可能にし、アフィン変換では補正しきれなかった細かいずれに対応します。ここでは、代表的なアルゴリズムである **SyN (Symmetric Normalization)** を使用します。

In [ ]:
from dipy.align.imwarp import SymmetricDiffeomorphicRegistration
from dipy.align.metrics import CCMetric

# 類似性指標として相互相関(CCMetric)を使用
# 2次元画像なので引数は2
metric = CCMetric(2, sigma_diff=20, radius=20)
sdr = SymmetricDiffeomorphicRegistration(metric)

# 最適化を実行。アフィン変換の結果を初期位置として与える(prealign)と、より安定で高速になります。
mapping = sdr.optimize(naomi1, naomi2, prealign=affine.affine)

# 画像2に非線形変換を適用
naomi2_warped = mapping.transform(naomi2)

# 結果の可視化
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
ax = axes.ravel()

ax[0].set_title('画像2 (非線形変換後)')
ax[0].imshow(naomi2_warped)

stereo_warped = np.zeros_like(stereo)
stereo_warped[..., 0] = naomi1
stereo_warped[..., 1] = naomi2_warped
ax[1].set_title('重ね合わせ (非線形変換)')
ax[1].imshow(stereo_warped)

fig.tight_layout()
plt.show()

非線形変換の結果、特に顔の周りのずれが大幅に改善され、ステレオ画像がより黄色くなった（＝一致度が高くなった）ことがわかります。

この変換が画像にどのような局所的な変形を加えたのかを可視化するために、**変形場（Displacement Field）**を見てみましょう。変形場は、各ピクセルがどれだけ移動したかを示します。変形がない場所は規則的なグリッドですが、局所的な歪みがある場所ではグリッドが歪んで見えます。

In [ ]:
from ndslib.viz import plot_diffeomorphic_map

fig, ax = plt.subplots(figsize=(5, 5))
plot_diffeomorphic_map(mapping, ax)
ax.imshow(naomi2_warped, alpha=0.4)
ax.set_title('変形場（グリッドの歪み）')
plt.show()

顔や肩のあたりでグリッドが歪んでいるのがわかります。これが、Diffeomorphic（微分同相写像）変換が作り出すことができる局所的な補正です。

## 実践2: 脳画像のレジストレーション

次に、この技術を3Dの脳画像データに適用してみましょう。ここでは、ある被験者のT1強調画像（個人の脳構造データ）を、**MNI152** という標準的な脳のテンプレートに位置合わせします。これは、異なる被験者のデータを同じ座標空間で比較・解析するために不可欠なステップです。

まず、必要なデータをダウンロードします。テンプレートデータは `templateflow` ライブラリを、個人の脳画像データは `ndslib` を使って取得します。

In [ ]:
# 必要なライブラリのインストール
!pip install templateflow nibabel ipywidgets

import templateflow.api as tflow
import nibabel as nib
from ndslib.data import download_bids_dataset
from nibabel.processing import resample_from_to

# MNIテンプレートの読み込み
mni_img = nib.load(tflow.get('MNI152NLin2009cAsym', resolution=1, suffix="T1w", desc=None))
mni_data = mni_img.get_fdata()

# 被験者のT1強調画像のダウンロードと読み込み
download_bids_dataset()
t1_img = nib.load("ds001233/sub-17/ses-pre/anat/sub-17_ses-pre_T1w.nii.gz")

# T1画像をMNIテンプレートの解像度と空間にリサンプリング
t1_resampled = resample_from_to(t1_img, (mni_img.shape, mni_img.affine))
t1_resamp_data = t1_resampled.get_fdata()

### インタラクティブなスライス表示

3Dの脳画像を比較するために、スライダーで矢状断（Sagittal）スライスを切り替えて表示できるインタラクティブなビューアを準備します。これにより、レジストレーションの効果を様々な断面で確認できます。

In [ ]:
from ipywidgets import interact, IntSlider, fixed

def show_slices(static, moving, title_moving, slice_idx, rot=True):
    """3つのパネルでスライスを可視化する関数"""
    # データを正規化してuint8に変換
    static_norm = (255 * (static / np.max(static))).astype(np.uint8)
    moving_norm = (255 * (moving / np.max(moving))).astype(np.uint8)

    # 表示用に画像を回転
    if rot:
        static_slice = np.rot90(static_norm[slice_idx, :, :])
        moving_slice = np.rot90(moving_norm[slice_idx, :, :])
    else:
        static_slice = static_norm[slice_idx, :, :]
        moving_slice = moving_norm[slice_idx, :, :]

    # ステレオ画像の作成
    stereo_shape = list(static_slice.shape) + [3]
    stereo = np.zeros(stereo_shape, dtype=np.uint8)
    stereo[..., 0] = static_slice
    stereo[..., 1] = moving_slice

    # プロット
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    ax = axes.ravel()
    ax[0].imshow(static_slice, cmap='gray')
    ax[0].set_title('MNIテンプレート (固定画像)')
    ax[0].axis('off')
    ax[1].imshow(moving_slice, cmap='gray')
    ax[1].set_title(title_moving)
    ax[1].axis('off')
    ax[2].imshow(stereo)
    ax[2].set_title('重ね合わせ (赤: MNI, 緑: T1)')
    ax[2].axis('off')
    fig.tight_layout()
    plt.show()

# レジストレーション前の状態を確認
slice_slider = IntSlider(min=0, max=mni_data.shape[0]-1, step=1, value=95, description='矢状断スライス:')
interact(
    show_slices, 
    static=fixed(mni_data), 
    moving=fixed(t1_resamp_data), 
    title_moving=fixed('被験者T1 (リサンプルのみ)'), 
    slice_idx=slice_slider
);

リサンプリングしただけでは、脳の位置や大きさが全く合っていないことがわかります。これから、3種類のレジストレーションを適用していきます。

### 1. 3D剛体変換

まずは3Dの剛体変換で、大まかな位置と向きを合わせます。

In [ ]:
from dipy.align.transforms import RigidTransform3D

affreg = AffineRegistration()
transform_rigid3d = RigidTransform3D()

rigid3d = affreg.optimize(mni_data, t1_resamp_data, transform_rigid3d, params0=None)
t1_rigid = rigid3d.transform(t1_resamp_data)

# 結果の確認
slice_slider = IntSlider(min=0, max=mni_data.shape[0]-1, step=1, value=95, description='矢状断スライス:')
interact(
    show_slices, 
    static=fixed(mni_data), 
    moving=fixed(t1_rigid), 
    title_moving=fixed('被験者T1 (剛体変換後)'), 
    slice_idx=slice_slider
);

### 2. 3Dアフィン変換

次に、脳の全体的なサイズの違いも補正するために、アフィン変換を適用します。

In [ ]:
from dipy.align.transforms import AffineTransform3D

transform_affine3d = AffineTransform3D()
affine3d = affreg.optimize(mni_data, t1_resamp_data, transform_affine3d, params0=None)
t1_affine = affine3d.transform(t1_resamp_data)

# 結果の確認
slice_slider = IntSlider(min=0, max=mni_data.shape[0]-1, step=1, value=95, description='矢状断スライス:')
interact(
    show_slices, 
    static=fixed(mni_data), 
    moving=fixed(t1_affine), 
    title_moving=fixed('被験者T1 (アフィン変換後)'), 
    slice_idx=slice_slider
);

アフィン変換により、脳のサイズ感もテンプレートにかなり近づきました。多くのアプリケーションではこれで十分な場合もあります。しかし、脳の輪郭（特に後頭部）や、内部の脳室の形、皮質の脳溝・脳回のパターンなど、細かい部分にはまだずれが見られます。

### 3. 3D非線形変換

最後に、非線形変換を用いて、脳溝・脳回レベルの細かい解剖学的構造の差異を吸収します。これにより、個人の脳を標準脳へ極めて精密にマッピングすることを目指します。

**注意:** 非線形レジストレーションは計算に時間がかかります（数分程度）。

In [ ]:
# 3D用の類似性指標を定義
metric = CCMetric(3) # 3次元なので引数は3
sdr = SymmetricDiffeomorphicRegistration(metric, level_iters=[10, 10, 5])

# アフィン変換の結果を初期値として非線形レジストレーションを実行
mapping = sdr.optimize(mni_data, t1_resamp_data, prealign=affine3d.affine)
t1_warped = mapping.transform(t1_resamp_data)

# 結果の確認
slice_slider = IntSlider(min=0, max=mni_data.shape[0]-1, step=1, value=95, description='矢状断スライス:')
interact(
    show_slices, 
    static=fixed(mni_data), 
    moving=fixed(t1_warped), 
    title_moving=fixed('被験者T1 (非線形変換後)'), 
    slice_idx=slice_slider
);

非線形変換の結果、アフィン変換では残っていた脳の輪郭や内部構造のずれが、さらに精密に補正されていることがわかります。重ね合わせ画像が全体的により黄色くなり、特に皮質のパターンがよく一致しています。

### 非線形変換の利点と注意点

**利点:**
*   **精密な対応付け:** 脳溝・脳回といった個人差の大きい解剖学的構造を高い精度で対応付けることができます。これにより、標準脳アトラスの情報を個人の脳に正確に適用したり、複数人のデータをボクセル単位で比較したりすることが可能になります。

**注意点:**
*   **計算コスト:** 非線形変換は自由度が高いため、計算に非常に時間がかかります。
*   **過剰な変形:** パラメータの設定によっては、解剖学的にありえないような不自然な変形を引き起こす可能性があります。例えば、あるべき構造を消してしまったり、ない構造を作り出してしまったりするリスクです。
*   **解釈の難しさ:** 非線形変換によって得られた結果（例えば、ある領域の体積の変化など）を解釈する際には、それが真の解剖学的差異なのか、レジストレーションのアーティファクトなのかを慎重に検討する必要があります。

このように、どのレジストレーション手法を選択するかは、研究の目的、必要な精度、そして許容できる計算コストによって決まります。

## まとめ

この章では、画像レジストレーションの基本的な考え方と、代表的な3つの手法（剛体変換、アフィン変換、非線形変換）を学びました。

- **剛体変換**は、位置と向きのずれを補正します。
- **アフィン変換**は、それに加えて全体的なサイズや傾きの違いを補正します。
- **非線形変換**は、局所的な形状の違いまで捉えることができ、最も精密な位置合わせを可能にしますが、計算コストと解釈には注意が必要です。

これらの画像処理アルゴリズムは、多くの脳画像解析パイプラインにおいて重要なステップを担っています。本章で得た直感と知識は、後続の章で学ぶ機械学習アルゴリズムを理解する上でも重要な基礎となります。